In [137]:
import serpent
from ethereum import tester as t, utils as u, abi

s = t.state()

In [138]:
serpent_vars_basic = '''
data x # Declare variable x. By default, x is numeric

def init():
    self.x = 42
    
def ask_x():
    return(self.x)

def incr_x():
    self.x = self.x + 1
    return(self.x)

'''

#s = t.state() # Commented out, since we don't need to reboot out test blockchain
c = s.abi_contract(serpent_vars_basic)

('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)


In [139]:
print(c.ask_x())
print(c.incr_x())
print(c.ask_x())

('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
42
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
43
('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
43


## Test accounts
Pyethereum defines in `tester` 10 dummy accounts, the first (actually 0th) of which is by default the sender of all contracts.

In [140]:
serpent_accts = '''
def ask_sender():
    return(msg.sender)
'''

c_accts = s.abi_contract(serpent_accts)

('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)


In [141]:
k0_int = c_accts.ask_sender()
print(k0_int)

('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 1000000000000000000000000L, 1)
745948140856946866108753121277737810491401257713


The function `ask_sender` returns a (bit) integer, which is some identifier of the dummy account `k0`. Let's test this, using variable names as in the University of Maryland [tutorial](http://mc2-umd.github.io/ethereumlab/docs/serpent_tutorial.pdf).

In [142]:
public_k0 = u.privtoaddr(t.k0)
public_k0

'\x82\xa9x\xb3\xf5\x96*[\tW\xd9\xee\x9e\xefG.\xe5[B\xf1'

What is this? Let's get to this another way, recalling that `t.ko` is the default message sender

In [143]:
k0_hex = hex(k0_int)
k0_hex

'0x82a978b3f5962a5b0957d9ee9eef472ee55b42f1L'

And in reverse:

In [144]:
int(0x82a978b3f5962a5b0957d9ee9eef472ee55b42f1L) == k0_int

True

To see what kind of a thing `public_k0` is as printed, note that tester `t` has a function `t.ascii_chr()`. If you are usind an IDE such as [PyCharm](https://www.jetbrains.com/pycharm/specials/pycharm/pycharm.html), then typing into iPython `t.` you will see a pop-up box with all methods associated with the pyethereum tester. That's how I found `t.ascii_chr()`. In Python, this is just the `chr()` command (see the Python documentation [here](https://docs.python.org/2/library/functions.html)).

You would think that `public_k0` should be the hex string of `hex(k0_int)` above, and this seems largely to be the case, but where do the other characters such as `*` and `[` fit in? It seems that `public_k0` is not the public key of account `t.k0`, but rather the hash of this account (private, public key, maybe a nonce in there).

It should not be just the hash of the public key, as can be seen below, since in blockchain transactions, we don't want things to depend only on the public key.

In [145]:
t.sha3(0x82a978b3f5962a5b0957d9ee9eef472ee55b42f1L)

'&\x01\xe8#E\xb8\x9d@\x02\x96\x83\xe1\xafI\xc0\xaa-\xacx\xbb\x94\xa4H`\x0f\x92hpp\x82K7'

In [146]:
chr(0x82)

'\x82'

In this case, the [source code](https://github.com/ethereum/pyethereum/blob/develop/ethereum/utils.py) for `u.privtoaddr()` helps us quite quickly to see what is actually going on

`def privtoaddr(x, extended=False):
    if len(x) > 32:
        x = decode_hex(x)
    o = sha3(privtopub(x)[1:])[12:]
return add_checksum(o) if extended else o`

First, `decode_hex` is a method imported from `rlp_utils`, and `privtopub` is imported from `bitcoin`, and then, as guessed at above, we hash the public key (starting at the 2nd character), and return the characters starting from 13 (note that `len(t.k0)` is 32, so we don't call `decode_hex` in this example).

Let's pull this function apart explicitly.

In [147]:
import bitcoin as bc
x = t.k0
x_public = bc.privtopub(x)
x_public


'\x04\xd6q\x04\xe6[a\xfc\xa1\xfcs\xa7Auf|\x11\xf1l\x03\x7f\xa8\xe4\x14I\x91\x89\xdf\x9dhpj\xcan\x1a\x1bN\xa0+p/\x10\x1e\xb6\t\xc4\xcd)d\xd4y\x00G\xa0dX\xba\x8a\x1a\x19\x04\n\xb1\xfe\r'

In [148]:
x_hash_inside = t.sha3(x_public[1:])
x_hash_inside

'\xb0m\xf7\xbd\xf0(2\xa7\xfb\xe6\x9ai\x82\xa9x\xb3\xf5\x96*[\tW\xd9\xee\x9e\xefG.\xe5[B\xf1'

In [149]:
x_hash_outside = x_hash_inside[12:]
x_hash_outside

'\x82\xa9x\xb3\xf5\x96*[\tW\xd9\xee\x9e\xefG.\xe5[B\xf1'

In [150]:
x_hash_outside == public_k0

True

In summary, `u.privtoaddr(x)` does not return the public key (so we will change our variable names soon), but rather the address (hash) of the account `x`, which requires both the private and public keys to generate (???). In the test environment here, these are pre-packaged into the test accounts `k0` to `k9`.

In [151]:
%xdel public_k0
addr_k0 = u.privtoaddr(t.k0)
addr_k0

'\x82\xa9x\xb3\xf5\x96*[\tW\xd9\xee\x9e\xefG.\xe5[B\xf1'

In [152]:
addr_k1 = u.privtoaddr(t.k1)
addr_k1

"}WzY{'B\xb4\x98\xcb\\\xf0\xc2l\xdc\xd7&\xd3\x9en"

In [153]:
addr_int_k1 = c_accts.ask_sender(sender = t.k1)
addr_k1_hex = hex(addr_int_k1)
addr_k1_hex

('b1', 1000000000000000000000000L)
('b2', 999999999999999996858408L)
('b3', 999999999999999999978666L, 1)


'0x7d577a597b2742b498cb5cf0c26cdcd726d39e6eL'